In [20]:
import hashlib
import hmac
import json
from time import time
from requests import post
from my_stuff import MufexKeys, MufexTestKeys


class Mufex:
    def __init__(self, symbol, api_key, secret_key, use_test_net: bool = False) -> None:
        if use_test_net:
            self.url_start = "https://api.testnet.mufex.finance"
        else:
            self.url_start = "https://api.mufex.finance"
        self.symbol = symbol
        self.api_key = api_key
        self.secret_key = secret_key

    def __HTTP_post_request(self, end_point, params):
        time_stamp = str(int(time() * 1000))
        params_as_string = self.__params_as_string(params=params)
        signature = self.__genSignature(time_stamp=time_stamp, paras_as_string=params_as_string)
        headers = {
            "MF-ACCESS-API-KEY": self.api_key,
            "MF-ACCESS-SIGN": signature,
            "MF-ACCESS-SIGN-TYPE": "2",
            "MF-ACCESS-TIMESTAMP": time_stamp,
            "MF-ACCESS-RECV-WINDOW": "5000",
            "Content-Type": "application/json",
        }
        response = post(
            url=self.url_start + end_point,
            headers=headers,
            data=params_as_string,
        )
        try:
            response_json = response.json()
            if response_json["message"] != "OK":
                raise KeyError(f' -> {response_json["message"]}')
        except KeyError as e:
            raise KeyError(f"{e}")
        return response_json

    def __params_as_string(self, params):
        params_as_string = str(json.dumps(params))
        return params_as_string

    def __genSignature(self, time_stamp, paras_as_string):
        param_str = time_stamp + self.api_key + "5000" + paras_as_string
        hash = hmac.new(bytes(self.secret_key, "utf-8"), param_str.encode("utf-8"), hashlib.sha256)
        return hash.hexdigest()

    def set_position_mode(self, mode):
        """
        https://www.mufex.finance/apidocs/derivatives/contract/index.html?console#t-dv_switchpositionmode
        Position mode. 0: One-Way mode. 3: Hedge mode
        """
        end_point = "/private/v1/account/set-position-mode"
        params = {
            "symbol": self.symbol,
            "mode": mode,
        }
        try:
            self.__HTTP_post_request(end_point=end_point, params=params)
        except KeyError as e:
            raise KeyError(f"Something is wrong setting postiion mode {e}")
    
    def create_order(
        self,
        params: dict,
        **vargs,
    ):
        """
        https://www.mufex.finance/apidocs/derivatives/contract/index.html#t-dv_placeorder
        """
        end_point = "/private/v1/trade/create"

        try:
            order_info = self.__HTTP_post_request(end_point=end_point, params=params)["data"]
            orderId = order_info["orderId"]
        except KeyError as e:
            raise KeyError(f"Something is wrong setting the limit entry order {e}")
        return orderId

    def create_long_entry_market_order(
        self,
        asset_amount: float,
        entry_client_id: str,
        time_in_force: str = "GoodTillCancel",
        **vargs,
    ):
        params = {
            "symbol": self.symbol,
            "positionIdx": 1,
            "side": "Buy",
            "orderType": "Market",
            "qty": str(asset_amount),
            "timeInForce": time_in_force,
            "orderLinkId": entry_client_id,
        }
        self.create_order(params=params)
    def create_long_entry_limit_order(
        self,
        asset_amount: float,
        entry_price: float,
        entry_client_id: str,
        time_in_force: str = "PostOnly",
        **vargs,
    ):
        params = {
            "symbol": self.symbol,
            "positionIdx": 1,
            "side": "Buy",
            "orderType": "Limit",
            "qty": str(asset_amount),
            "price": str(entry_price),
            "timeInForce": time_in_force,
            "orderLinkId": entry_client_id,
        }
        self.create_order(params=params)
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret,
    symbol="BTCUSDT",
)
mufex_test = Mufex(
    api_key=MufexTestKeys.api_key,
    secret_key=MufexTestKeys.secret,
    symbol="BTCUSDT",
    use_test_net=True
)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
mufex_main.set_position_mode(mode=3)

KeyError: 'Something is wrong setting postiion mode "\' -> position mode not modified\'"'

In [26]:
mufex_test.set_position_mode(mode=3)

KeyError: 'Something is wrong setting postiion mode "\' -> position mode not modified\'"'

In [24]:
mufex_test.set_position_mode(mode=0)

In [15]:
mufex_test.create_long_entry_market_order(
    asset_amount=0.001,
    entry_client_id="sdfwer43523235253"
)

In [18]:
mufex_test.create_long_entry_limit_order(
    asset_amount=0.001,
    entry_client_id="561981651681",
    entry_price=27000,
)

In [19]:
mufex_main.create_long_entry_limit_order(
    asset_amount=0.001,
    entry_client_id="561981651681",
    entry_price=27000,
)

KeyError: 'Something is wrong setting the limit entry order "\' -> upstream server error\'"'